In [1]:
import tensorflow as tf
from tensorflow.contrib.cudnn_rnn import CudnnCompatibleLSTMCell, CudnnLSTM
import numpy as np

In [2]:
seq_lens = [3, 5, 7]
values = tf.constant(np.random.rand(7, 3, 4), dtype=tf.float32)
cell = CudnnLSTM(num_layers=1, num_units=2, input_mode='linear_input', direction='unidirectional', dropout=0.0)
cell.build((None, None, 7))

In [3]:
zeros = tf.zeros((1,3,2))
zeros_state = (zeros, zeros)
max_val = tf.constant(-1e6, shape=(3, 2))
mean_val = tf.constant(0.0, shape=(3, 2))

In [4]:
def cond(i, inputs, state, sl, max_val, mean_val, bptt, max_len): return i < max_len
def body(i, inputs, state, sl, max_val, mean_val, bptt, max_len):
    i_to = tf.minimum(i+bptt, max_len)
    slice_inputs = inputs[i:i_to]
    outputs, next_state = cell.call(slice_inputs, state)
    mask = tf.expand_dims(tf.transpose(tf.sequence_mask(tf.minimum(sl-i, bptt), dtype=tf.float32), (1, 0)), axis=-1)
    max_outputs = outputs * mask + (1 - mask) * -1e6
    max_val = tf.maximum(max_val, tf.reduce_max(max_outputs, axis=0))
    mean_outputs = outputs * mask
    mean_val = (mean_val * tf.expand_dims(tf.to_float(tf.minimum(i, sl)), axis=-1) + tf.reduce_sum(mean_outputs, axis=0)) / tf.expand_dims(tf.to_float(tf.minimum(i_to, sl)), axis=-1)
    return i_to, inputs, next_state, sl, max_val, mean_val, bptt, max_len

In [5]:
_, _, _, _, max_val, mean_val, _, _ = tf.while_loop(cond, body, [0, values, zeros_state, seq_lens, max_val, mean_val, 2, 7])

In [6]:
var = cell.variables[0]
var

<tf.Variable 'cudnn_lstm/opaque_kernel:0' shape=<unknown> dtype=float32_ref>

In [7]:
grad_max = tf.gradients(max_val, [var])

In [8]:
sess=tf.Session()

In [9]:
sess.run(tf.global_variables_initializer())

In [11]:
sess.run([max_val, mean_val, grad_max])

[array([[0.1769983 , 0.26831168],
        [0.148014  , 0.27984455],
        [0.20452362, 0.27956623]], dtype=float32),
 array([[0.1429977 , 0.22149013],
        [0.10645799, 0.25378737],
        [0.12940875, 0.24346709]], dtype=float32),
 [array([0.03406822, 0.04981756, 0.06814141, 0.05405072, 0.17690864,
         0.13425422, 0.20134673, 0.13496011, 0.04209283, 0.0481848 ,
         0.07238872, 0.05868972, 0.10773583, 0.07703532, 0.12508363,
         0.10048503, 0.03387313, 0.05545799, 0.07734708, 0.05616804,
         0.16275106, 0.14997071, 0.21757352, 0.17776048, 0.07907394,
         0.22262707, 0.33526558, 0.26234317, 0.09869855, 0.04255045,
         0.12463032, 0.13958623, 0.00915636, 0.01765727, 0.04070983,
         0.07395065, 0.01133056, 0.0232649 , 0.03026436, 0.05538224,
         0.01023651, 0.01836793, 0.04759242, 0.08308448, 0.04463354,
         0.08591186, 0.03678876, 0.06745024, 0.08966027, 0.3397737 ,
         0.0954781 , 0.22252406, 0.09837294, 0.36225098, 0.3875872 ,
   